This script produces the outputs for the events article.

<b> This script relies on the csv's made in the '1_seasonal_adjustment.ipynb' notebook. </b> Make sure that script has run before running this script.

In [ ]:
project_path = "/home/jupyter"
import os
import sys
sys.path.append(project_path)
sys.path.append(f'{project_path}/ft_events/src/utils')

from google.cloud import bigquery
from google.cloud import storage

import importlib

import numpy as np
import pandas as pd
from plotly import graph_objs as go
import seaborn as sns
import geopandas as gpd

import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import plotly.express as px
import ipywidgets as widgets

from fintrans_toolbox.src import table_utils as t
from fintrans_toolbox.src import bq_utils as bq

client = bigquery.Client()


In [ ]:
# UK wide dfs for adjusting spend by UK cardholders
sql_all_spend_sml = f"""SELECT time_period_value, merchant_location, spend, transactions, cardholders
  FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
  WHERE time_period = 'Month' AND 
  merchant_location_level = 'All' AND
  cardholder_issuing_level = 'All' AND
  mcg = 'All'  AND 
  time_period_value <= '202503'
  ORDER BY time_period_value, merchant_location"""

sml_hl = client.query(sql_all_spend_sml).to_dataframe()
sml_hl = t.create_date_time(sml_hl)

# Find UK wide index cardholders
sml_hl["idx_cards"] = sml_hl["cardholders"].transform(lambda x: x / x.iloc[0])
sml_hl["idx_spend"] = sml_hl["spend"] / sml_hl["idx_cards"]

In [ ]:
def add_covid_periods(fig):
    """Add shaded regions for the covid periods.

    Args:
      fig (plotly.graph.objs._figure.Figure): The figure to add shaded regions to.

    Returns (plotly.graph.objs._figure.Figure):
      The shaded region object to be plotted.
    """
    # define covid lockdown start and end dates
    covid_periods = [
        ("2020-03-23", "2020-06-15"),
        ("2020-11-05", "2020-12-02"),
        ("2021-01-04", "2021-03-29"),
    ]
    # change colour of the covid lockdowns
    fillcolor = "gainsboro"

    # add a dummy trace so that we can add the covid periods to the legend
    # add markers to square to make it show as a square in the legend.
    fig.add_trace(
        go.Scatter(
            x=[None],
            y=[None],
            mode="markers",
            marker=dict(color=fillcolor, symbol="square"),
            name="Covid Lockdowns",
        )
    )

    # add covid periods to plot
    for start_date, end_date in covid_periods:
        fig.add_shape(
            type="rect",
            xref="x",
            yref="paper",
            x0=start_date,
            y0=0,
            x1=end_date,
            y1=1,
            fillcolor=fillcolor,
            opacity=0.5,
            layer="below",
            line_width=0,
        )

    return fig

In [ ]:
def location_level_validation(df, merch_card):
    
    # Counts number of numerals in each location code
    df["number"] = df[f'{merch_card}'].str.extract(
        "(\d+)", expand=False
    )

    if len(df[f'{merch_card.split("_")[0]}_location_level'].unique()) >1:

        first_df = df[df[f'{merch_card.split("_")[0]}_location_level'] == 'POSTAL_AREA'].copy()
        second_df = df[df[f'{merch_card.split("_")[0]}_location_level'] == 'POSTAL_DISTRICT'].copy()

        # no nums in postal area
        first_df = first_df.loc[first_df["number"].isna() == True].drop(
            "number", axis=1
        )

        # nums in postal district
        second_df = second_df.loc[second_df["number"].isna() == False].drop(
            "number", axis=1
        )


        clean_df = pd.concat([first_df, second_df])

    else:
        if df[f'{merch_card.split("_")[0]}_location_level'][1] == 'POSTAL_AREA':

            clean_df = df.loc[df["number"].isna() == True].drop(
                "number", axis=1
            )

        if df[f'{merch_card.split("_")[0]}_location_level'][1] == 'POSTAL_DISTRICT':

            clean_df = df.loc[df["number"].isna() == False].drop(
                "number", axis=1
            )

    return clean_df

In [ ]:
all_areas_uk_colours = {'UK': '#003c57',
 'CF': '#a8bd3a',
 'EH': '#27a0cc',
 'HA': '#0f8243',
 'L': '#F46A25',
'CF10': '#a8bd3a',
 'EH12': '#27a0cc',
 'HA9': '#0f8243',
 'L4': '#F46A25',
                       'CF': '#a8bd3a',
 'EH': '#27a0cc',
 'TW': '#F46A25',
                       'CF10': '#a8bd3a',
 'EH12': '#27a0cc',
 'TW2': '#F46A25'}

# TS

In [ ]:
def add_ts_periods(fig):

    concert_periods = [
        ("2024-05-20", "2024-06-10"),
    ]
    fillcolor = "grey"

    fig.add_trace(
        go.Scatter(
            x=[None],
            y=[None],
            mode="markers",
            marker=dict(color=fillcolor, symbol="square"),
            name="TS Concert",
        )
    )

    for start_date, end_date in concert_periods:
        fig.add_shape(
            type="rect",
            xref="x",
            yref="paper",
            x0=start_date,
            y0=0,
            x1=end_date,
            y1=1,
            fillcolor=fillcolor,
            opacity=0.5,
            layer="below",
            line_width=0,
        )

    return fig

In [ ]:
def add_extra_wembley(fig):

    concert_periods = [
        ("2024-07-20", "2024-08-10"),
    ]
    fillcolor = "gainsboro"

    fig.add_trace(
        go.Scatter(
            x=[None],
            y=[None],
            mode="markers",
            marker=dict(color=fillcolor, symbol="square"),
            name="Extra Wembley (HA)",
        )
    )

    for start_date, end_date in concert_periods:
        fig.add_shape(
            type="rect",
            xref="x",
            yref="paper",
            x0=start_date,
            y0=0,
            x1=end_date,
            y1=1,
            fillcolor=fillcolor,
            opacity=0.5,
            layer="below",
            line_width=0,
        )

    return fig

In [ ]:
def calc_index_yoy(df, need_date_cols, group_list):
    
    result = df.copy()
    if need_date_cols is True:
        result['year'] = result['date_time'].dt.year
        result['month'] = result['date_time'].dt.month
    
    metrics = ['spend',]
    month_group = group_list + ['month']

    result['yoy_spend'] = result.groupby(month_group)['spend'].diff(periods=1)
    result['yoy_spend_perc'] = result.groupby(month_group)['spend'].pct_change(periods=1)*100

    return result

In [ ]:
ts_pas = ['EH', 'L', 'CF', 'HA']
ts_pds = ('EH12', 'L4', 'CF10', 'HA9')

ts_all_codes = ['EH', 'L', 'CF', 'HA', 'EH12', 'L4', 'CF10', 'HA9']

areas_uk_colours = {'UK': '#003c57',
 'CF': '#a8bd3a',
 'EH': '#27a0cc',
 'HA': '#0f8243',
 'L': '#F46A25'}

all_areas_uk_colours = {'UK': '#003c57',
 'CF': '#a8bd3a',
 'EH': '#27a0cc',
 'HA': '#0f8243',
 'L': '#F46A25',
'CF10': '#a8bd3a',
 'EH12': '#27a0cc',
 'HA9': '#0f8243',
 'L4': '#F46A25',
                       'CF': '#a8bd3a',
 'EH': '#27a0cc',
 'TW': '#F46A25',
                       'CF10': '#a8bd3a',
 'EH12': '#27a0cc',
 'TW2': '#F46A25'}

### SEASONALLY ADJUSTED: High-level

In [ ]:
ts_high_level = pd.read_csv("ts_high_level.csv")

# Re-shaping and re-naming the data
ts_hl_long = ts_high_level.melt(id_vars='Date', var_name='merchant_location', value_name = 'spend')
ts_hl_long['Date'] = pd.to_datetime(ts_hl_long['Date'], format='%Y-%m-%d')
ts_hl_long.rename(columns={"Date": "date_time"}, inplace = True)

# Calculating year-on-year changes
ts_hl_long = calc_index_yoy(ts_hl_long, need_date_cols = True, group_list = ['merchant_location'])

In [ ]:
fig = px.line(
ts_hl_long,
x="date_time",
y="spend",
color = 'merchant_location',
title=f"Index spend at venue postal district",
height = 500,
template = 'simple_white',
color_discrete_map=all_areas_uk_colours

)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.011,
    xanchor="right",
    x=1
))

fig.update_layout(legend_title_text=f'Venue postal district:', 
                  title_subtitle_text=f'MCG = All | Seasonally adjusted',
                  yaxis_title = f'Index sum spend',
                 xaxis_title = 'Date')
fig.update_traces(selector=dict(name='UK'), line=dict(dash='dash'))  #dashed line for UK

fig = add_ts_periods(fig)
fig = add_extra_wembley(fig)
fig = add_covid_periods(fig)

fig.show()

In [ ]:
fig = px.line(
ts_hl_long,
x="date_time",
y="yoy_spend_perc",
color = 'merchant_location',
title=f"YOY spend at venue postal district",
height = 500,
template = 'simple_white',
color_discrete_map=all_areas_uk_colours

)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.update_layout(legend_title_text=f'Venue postal district:', 
                  title_subtitle_text=f'MCG = All | Seasonally adjusted',
                  yaxis_title = f'Index sum spend',
                 xaxis_title = 'Date')
fig.update_traces(selector=dict(name='UK'), line=dict(dash='dash'))  #dashed line for UK

fig = add_ts_periods(fig)
fig = add_extra_wembley(fig)


fig.show()

### NON-SEASONALLY ADJUSTED: High level

In [ ]:
ts_pds = ('EH12', 'L4', 'CF10', 'HA9')

sql_spend = f"""SELECT *
FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
WHERE time_period = 'Month' AND
mcg = 'All' AND
mcc = 'All' AND
merchant_location_level = 'POSTAL_DISTRICT' AND
cardholder_issuing_level = 'All' AND
merchant_location IN {ts_pds} AND
time_period_value <= '202503'
ORDER BY time_period_value, merchant_location_level, cardholder_issuing_country, merchant_location, mcg, mcc, spend, transactions, cardholders"""

sml_df = client.query(sql_spend).to_dataframe()
ts_df = t.create_date_time(sml_df)
ts_df = location_level_validation(df = ts_df, merch_card = 'merchant_location')

In [ ]:
# Merging UK wide SML index cardholders
ts_df = ts_df.merge(sml_hl[['time_period_value', 'idx_cards']], on='time_period_value', how='left')
# Adjusting spend by UK cardholders
ts_df["idx_spend"] = ts_df["spend"] / ts_df["idx_cards"]

In [ ]:
# Filtering to get venue and UK sums
venue_df = ts_df[(ts_df['merchant_location'].isin(ts_pds))].groupby(['date_time', 'merchant_location']).agg({"spend" : "sum", "idx_spend" : "sum", "transactions" : "sum", "cardholders" : "sum"}).reset_index()
uk_df = sml_hl.groupby(['date_time']).agg({"spend" : "sum", "idx_spend" : "sum", "transactions" : "sum", "cardholders" : "sum"}).reset_index()
uk_df['merchant_location'] = 'UK'

full_ts_df = pd.concat([venue_df, uk_df])

In [ ]:
metrics = ['spend', 'idx_spend', 'transactions', 'cardholders']
month_group = ['merchant_location', 'month']
group_list = ['merchant_location']

full_ts_df['year'] = full_ts_df['date_time'].dt.year
full_ts_df['month'] = full_ts_df['date_time'].dt.month

for i in metrics:
    # calc year-on-year differences
    full_ts_df[f'yoy_{i}'] = full_ts_df.groupby(month_group)[f'{i}'].diff(periods=1)

    # calc year-on-year % change
    full_ts_df[f'yoy_{i}_perc'] = full_ts_df.groupby(month_group)[f'{i}'].pct_change(periods=1)*100

    # index to jan 2019
    full_ts_df[f"index_{i}"] = full_ts_df.groupby(group_list)[f"{i}"].transform(
    lambda x: x / x.iloc[0]
)

In [ ]:
fig = px.line(
full_ts_df,

x="date_time",
y="index_idx_spend",
color = 'merchant_location',
title=f"Index card-adj spend at postal districts",
height = 500,
template = 'simple_white',
color_discrete_map=all_areas_uk_colours

)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.0,
    xanchor="right",
    x=1
))

fig.update_layout(legend_title_text=f'Venue postal district:', 
                  title_subtitle_text=f'MCG = All',
                  yaxis_title = f'Index sum spend',
                 xaxis_title = 'Date')
fig.update_traces(selector=dict(name='UK'), line=dict(dash='dash'))  #dashed line for UK

fig = add_ts_periods(fig)
fig = add_extra_wembley(fig)
fig = add_covid_periods(fig)

fig.show()

In [ ]:
fig = px.line(
full_ts_df,

x="date_time",
y="index_spend",
color = 'merchant_location',
title=f"Index spend at postal districts",
height = 500,
template = 'simple_white',
color_discrete_map=all_areas_uk_colours

)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.0,
    xanchor="right",
    x=1
))

fig.update_layout(legend_title_text=f'Venue postal district:', 
                  title_subtitle_text=f'MCG = All | Non-cardholder adjusted',
                  yaxis_title = f'Index sum spend',
                 xaxis_title = 'Date')
fig.update_traces(selector=dict(name='UK'), line=dict(dash='dash'))  #dashed line for UK

fig = add_ts_periods(fig)
fig = add_extra_wembley(fig)
fig = add_covid_periods(fig)

fig.show()

### SEASONALLY ADJUSTED: International spend

In [ ]:
ts_international = pd.read_csv("ts_international_spend.csv")

# Re-shaping and re-naming the data
ts_international = ts_international.melt(id_vars='Date', var_name='merchant_location', value_name = 'spend')
ts_international['Date'] = pd.to_datetime(ts_international['Date'], format='%Y-%m-%d')
ts_international.rename(columns={"Date": "date_time"}, inplace = True)

# Calculating year-on-year changes
ts_international = calc_index_yoy(ts_international, need_date_cols = True, group_list = ['merchant_location'])

In [ ]:
fig = px.line(
ts_international,
x="date_time",
y="spend",
color = 'merchant_location',
title=f"Index international spend at venue postal district",
height = 500,
template = 'simple_white',
color_discrete_map=all_areas_uk_colours

)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.update_layout(legend_title_text=f'Venue postal district:', 
                  title_subtitle_text=f'MCG = All | Seasonally adjusted',
                  yaxis_title = f'Index sum international spend',
                 xaxis_title = 'Date')
fig.update_traces(selector=dict(name='UK'), line=dict(dash='dash'))  #dashed line for UK

fig = add_ts_periods(fig)
fig = add_extra_wembley(fig)


fig.show()

In [ ]:
fig = px.line(
ts_international,
x="date_time",
y="yoy_spend_perc",
color = 'merchant_location',
title=f"YOY international spend at venue postal district",
height = 500,
template = 'simple_white',
color_discrete_map=all_areas_uk_colours

)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.update_layout(legend_title_text=f'Venue postal district:', 
                  title_subtitle_text=f'MCG = All | Seasonally adjusted',
                  yaxis_title = f'Index sum international spend',
                 xaxis_title = 'Date')
fig.update_traces(selector=dict(name='UK'), line=dict(dash='dash'))  #dashed line for UK

fig = add_ts_periods(fig)
fig = add_extra_wembley(fig)


fig.show()

### NON-SEASONALLY ADJUSTED: International spend

In [ ]:
sql = f"""SELECT *
  FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
  WHERE time_period = 'Month' AND 
  merchant_location_level = 'All' AND
  cardholder_issuing_level = 'International' AND
  cardholder_issuing_country = 'All' AND
  mcg = 'All' AND
  time_period_value <= '202503'
  ORDER BY time_period_value, merchant_location
  """

int_ts_uk = client.query(sql).to_dataframe()
int_ts_uk = t.create_date_time(int_ts_uk)

# Find UK wide index cardholders
int_ts_uk["idx_cards"] = int_ts_uk["cardholders"].transform(lambda x: x / x.iloc[0])
int_ts_uk["idx_spend"] = int_ts_uk["spend"] / int_ts_uk["idx_cards"]

In [ ]:
client = bigquery.Client()

sql = f"""SELECT *
  FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
  WHERE time_period = 'Month' AND 
  merchant_location_level = 'POSTAL_DISTRICT' AND
  cardholder_issuing_level = 'International' AND
  cardholder_issuing_country = 'All' AND
  mcg = 'All' AND
  merchant_location IN  {ts_pds} AND
  time_period_value <= '202503'
  ORDER BY time_period_value, merchant_location
  """

df_full_int = client.query(sql).to_dataframe()
df_full_int = t.create_date_time(df_full_int)

In [ ]:
# Merging UK wide SML index cardholders
df_full_int = df_full_int.merge(int_ts_uk[['time_period_value', 'idx_cards']], on='time_period_value', how='left')
# Adjusting spend by UK cardholders
df_full_int["idx_spend"] = df_full_int["spend"] / df_full_int["idx_cards"]

In [ ]:
month_group = ['merchant_location', 'month']
group = ['merchant_location']

df_full_int = df_full_int.sort_values(['date_time']).reset_index(drop=True)
# index spend
df_full_int["index_spend"] = df_full_int.groupby(group)["spend"].transform(
    lambda x: x / x.iloc[0]
)
# index adj spend
df_full_int["index_adj_spend"] = df_full_int.groupby(group)["idx_spend"].transform(
    lambda x: x / x.iloc[0]
)
# index cardholders
df_full_int["index_cardholders"] = df_full_int.groupby(group)["cardholders"].transform(
    lambda x: x / x.iloc[0]
)

# month-on-month
df_full_int['mm_perc_spend'] = df_full_int.groupby(group)['index_spend'].pct_change(periods=1)*100
df_full_int['mm_perc_cardholders'] = df_full_int.groupby(group)['index_cardholders'].pct_change(periods=1)*100

# year-on-year
df_full_int['yy_perc_spend'] = df_full_int.groupby(month_group)['index_spend'].pct_change(periods=1)*100
df_full_int['yy_perc_adj_spend'] = df_full_int.groupby(month_group)['index_adj_spend'].pct_change(periods=1)*100

df_full_int['yy_perc_cardholders'] = df_full_int.groupby(month_group)['index_cardholders'].pct_change(periods=1)*100



In [ ]:
fig = px.line(
df_full_int,
x="date_time",
y = "index_adj_spend",
template='simple_white',
color = 'merchant_location',
height = 500,
width = 1000,
title = 'Index international spend')

fig = add_ts_periods(fig)

fig = add_extra_wembley(fig)

fig.show()

-----------------------------

-----------------------------

# SN

In [ ]:
def add_nations_periods(fig):

    # define covid lockdown start and end dates
    game_periods = [
        ("2019-02-01", "2019-03-16"),
        ("2020-02-01", "2020-03-08"),
        ("2021-02-06", "2021-03-26"),
        ("2022-02-01", "2022-03-01"),
        ("2023-02-01", "2023-03-01"),
        ("2024-02-01", "2024-03-01"),
        ("2025-02-01", "2025-03-15"),
    ]
    # change colour of the covid lockdowns
    fillcolor = "grey"

    # add a dummy trace so that we can add the covid periods to the legend
    # add markers to square to make it show as a square in the legend.
    fig.add_trace(
        go.Scatter(
            x=[None],
            y=[None],
            mode="markers",
            marker=dict(color=fillcolor, symbol="square"),
            name="Six Nations",
        )
    )

    # add covid periods to plot
    for start_date, end_date in game_periods:
        fig.add_shape(
            type="rect",
            xref="x",
            yref="paper",
            x0=start_date,
            y0=0,
            x1=end_date,
            y1=1,
            fillcolor=fillcolor,
            opacity=0.5,
            layer="below",
            line_width=0,
        )

    return fig

In [ ]:
sn_pds = ('EH12', 'CF10', 'TW2')
sn_countries = ('FRANCE', 'ITALY', 'REPUBLIC OF IRELAND')

### SEASONALLY ADJUSTED: High level

In [ ]:
sn_high_level = pd.read_csv("sn_high_level.csv")

# Re-shaping and re-naming the data
sn_high_level = sn_high_level.melt(id_vars='Date', var_name='merchant_location', value_name = 'spend')
sn_high_level['Date'] = pd.to_datetime(sn_high_level['Date'], format='%Y-%m-%d')
sn_high_level.rename(columns={"Date": "date_time"}, inplace = True)

# Calculating year-on-year changes
sn_high_level = calc_index_yoy(sn_high_level, need_date_cols = True, group_list = ['merchant_location'])

In [ ]:
fig = px.line(
sn_high_level,
x="date_time",
y="spend",
color = 'merchant_location',
title=f"Index spend at venue postal district",
height = 500,
template = 'simple_white',
color_discrete_map=all_areas_uk_colours

)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.update_layout(legend_title_text=f'Venue postal district:', 
                  title_subtitle_text=f'MCG = All | Seasonally adjusted',
                  yaxis_title = f'Index sum spend',
                 xaxis_title = 'Date')
fig.update_traces(selector=dict(name='UK'), line=dict(dash='dash'))  #dashed line for UK
fig = add_nations_periods(fig)


fig.show()

### SEASONALLY ADJUSTED: International spend

In [ ]:
sn_int_colours = {'France': '#3E8ABF',
 'Italy': '#FF861B',
 'Ireland': '#1B981B',
  'FRANCE': '#3E8ABF',
 'ITALY': '#FF861B',
 'REPUBLIC OF IRELAND': '#1B981B'}

In [ ]:
matches_log = pd.DataFrame(
    # Cardiff
    [{'date_time':'2019-03-01', 'area': 'CF', 'district': 'CF10', 'country': 'Ireland'},
    {'date_time':'2020-02-01', 'area': 'CF', 'district': 'CF10',  'country': 'Italy'},
    {'date_time':'2020-02-01', 'area': 'CF', 'district': 'CF10',   'country': 'France'},
    {'date_time':'2022-03-01', 'area': 'CF', 'district': 'CF10',   'country': 'France'},
    {'date_time':'2022-03-01', 'area': 'CF', 'district': 'CF10',   'country': 'Italy'},
    {'date_time':'2023-02-01', 'area': 'CF', 'district': 'CF10',   'country': 'Ireland'},
    {'date_time':'2024-03-01', 'area': 'CF', 'district': 'CF10',   'country': 'France'},
    {'date_time':'2024-03-01', 'area': 'CF', 'district': 'CF10',   'country': 'Italy'},
    {'date_time':'2025-02-01', 'area': 'CF', 'district': 'CF10',   'country': 'Ireland'},
     
    # Edinburgh
    {'date_time':'2019-02-01', 'area': 'EH', 'district': 'EH12',   'country': 'Italy'},
    {'date_time':'2019-02-01', 'area': 'EH', 'district': 'EH12', 'country': 'Ireland'},
    {'date_time':'2020-03-01', 'area': 'EH', 'district': 'EH12', 'country': 'France'},
    {'date_time':'2022-02-01', 'area': 'EH', 'district': 'EH12', 'country': 'France'},
    {'date_time':'2023-03-01', 'area': 'EH', 'district': 'EH12', 'country': 'Ireland'},
    {'date_time':'2023-03-01', 'area': 'EH', 'district': 'EH12', 'country': 'Italy'},
    {'date_time':'2024-02-01', 'area': 'EH', 'district': 'EH12', 'country': 'France'},
    {'date_time':'2025-02-01', 'area': 'EH', 'district': 'EH12', 'country': 'Italy'},
    {'date_time':'2025-02-01', 'area': 'EH', 'district': 'EH12', 'country': 'Ireland'},
    
    # Twickenham
    {'date_time':'2019-02-01', 'area': 'TW', 'district': 'TW2', 'country': 'France'},
    {'date_time':'2019-03-01', 'area': 'TW', 'district': 'TW2', 'country': 'Italy'},
    {'date_time':'2020-02-01', 'area': 'TW', 'district': 'TW2', 'country': 'Ireland'},
    {'date_time':'2022-03-01', 'area': 'TW', 'district': 'TW2', 'country': 'Ireland'},
    {'date_time':'2023-02-01', 'area': 'TW', 'district': 'TW2', 'country': 'Italy'},
    {'date_time':'2023-03-01', 'area': 'TW', 'district': 'TW2', 'country': 'France'},
    {'date_time':'2024-03-01', 'area': 'TW', 'district': 'TW2', 'country': 'Ireland'},
    {'date_time':'2025-02-01', 'area': 'TW', 'district': 'TW2', 'country': 'France'},
    {'date_time':'2025-03-01', 'area': 'TW', 'district': 'TW2', 'country': 'Italy'},])


matches_log['game_name'] = 'v ' + matches_log['country']
area_to_host = {'CF': 'Wales', 'TW': 'England', 'EH': 'Scotland'}

matches_log['host_name'] = matches_log['area'].map(area_to_host)


In [ ]:
sn_international = pd.read_csv("sn_international_spend.csv")

# Re-shaping and re-naming the data
sn_international = sn_international.melt(id_vars='Date', var_name='merchant_location', value_name = 'spend')
sn_international['Date'] = pd.to_datetime(sn_international['Date'], format='%Y-%m-%d')
sn_international.rename(columns={"Date": "date_time"}, inplace = True)

# Calculating year-on-year changes
sn_international = calc_index_yoy(sn_international, need_date_cols = True, group_list = ['merchant_location'])

In [ ]:
fig = px.line(
sn_international,
x="date_time",
y="spend",
color = 'merchant_location',
title=f"CF10 international spend",
height = 500,
template = 'simple_white',
color_discrete_map=sn_int_colours

)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.update_layout(legend_title_text=f'Venue postal district:', 
                  title_subtitle_text=f'MCG = All | Seasonally adjusted',
                  yaxis_title = f'Index sum international spend',
                 xaxis_title = 'Date')
fig.update_traces(selector=dict(name='UK'), line=dict(dash='dash'))  #dashed line for UK

fig = add_nations_periods(fig)

# Filter matches_log for the current area
matches_in_area = matches_log[matches_log['district'] == 'CF10']

for _, match in matches_in_area.iterrows():
    try:
        # Extract spend value for the specific date_time and cardholder_issuing_country
        spend_value = sn_international[
            (sn_international['date_time'] == match['date_time']) & 
            (sn_international['merchant_location'] == match['country'])
        ]['spend'].iloc[0]

        # Adding vertical line at the match date
        fig.add_vline(
            x=match['date_time'],
            line_dash='dot',
            line_color='gray',
            opacity=0.6
        )

        # Adding text annotations at the corresponding 'spend' value
        fig.add_annotation(
            x=match['date_time'],  # Position the annotation at the same x position
            y=spend_value,  # Use the extracted spend value as the y position
            text=match['host_name'] + ' v '+  match['game_name'][2:].title(),
            font=dict(size=10, color="black",style="italic"),  # Font style of the annotation
            align='center',  # Text alignment,
        )
    except IndexError: 
        pass


fig.show()

In [ ]:
fig = px.line(
sn_international,
x="date_time",
y="yoy_spend_perc",
color = 'merchant_location',
title=f"Year on year CF10 international spend",
height = 500,
template = 'simple_white',
color_discrete_map=sn_int_colours

)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.update_layout(legend_title_text=f'Venue postal district:', 
                  title_subtitle_text=f'MCG = All | Seasonally adjusted',
                  yaxis_title = f'Index sum international spend',
                 xaxis_title = 'Date')
fig.update_traces(selector=dict(name='UK'), line=dict(dash='dash'))  #dashed line for UK

fig = add_nations_periods(fig)

# Filter matches_log for the current area
matches_in_area = matches_log[matches_log['district'] == 'CF10']

for _, match in matches_in_area.iterrows():
    try:
        # Extract spend value for the specific date_time and cardholder_issuing_country
        spend_value = sn_international[
            (sn_international['date_time'] == match['date_time']) & 
            (sn_international['merchant_location'] == match['country'])
        ]['yoy_spend_perc'].iloc[0]

        # Adding vertical line at the match date
        fig.add_vline(
            x=match['date_time'],
            line_dash='dot',
            line_color='gray',
            opacity=0.6
        )

        # Adding text annotations at the corresponding 'spend' value
        fig.add_annotation(
            x=match['date_time'],  # Position the annotation at the same x position
            y=spend_value,  # Use the extracted spend value as the y position
            text=match['host_name'] + ' v '+  match['game_name'][2:].title(),
            font=dict(size=10, color="black",style="italic"),  # Font style of the annotation
            align='center',  # Text alignment,
        )
    except IndexError: 
        pass


fig.show()

### NON-SEASONALLY ADJUSTED: International spend

In [ ]:
sql = f"""SELECT *
  FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
  WHERE time_period = 'Month' AND 
  merchant_location_level = 'All' AND
  cardholder_issuing_level = 'International' AND
  cardholder_issuing_country = 'All' AND
  mcg = 'All' AND
  time_period_value <= '202503'
  ORDER BY time_period_value, merchant_location
  """

int_hl = client.query(sql).to_dataframe()
int_hl = t.create_date_time(int_hl)

# Adjust spend to cardholders
int_hl["idx_cards"] = int_hl["cardholders"].transform(lambda x: x / x.iloc[0])

# spend / index
int_hl["idx_spend"] = int_hl["spend"] / int_hl["idx_cards"]

In [ ]:
sql_int_spend= f"""SELECT *
FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
WHERE time_period = 'Month' AND
mcg = 'All' AND
merchant_location_level != 'All' AND
cardholder_issuing_level != 'All' AND
merchant_location_level = 'POSTAL_DISTRICT' AND
merchant_location = 'CF10' AND
time_period_value <= '202503' AND
cardholder_issuing_country IN {sn_countries}
ORDER BY time_period_value"""
df_sn_int = client.query(sql_int_spend).to_dataframe()
df_sn_int = t.create_date_time(df_sn_int)

In [ ]:
df_sn_int = df_sn_int.merge(int_hl[['time_period_value', 'idx_cards']], on='time_period_value', how='left')


In [ ]:
df_sn_int["idx_spend"] = df_sn_int["spend"] / df_sn_int["idx_cards"]

metrics = ['spend', 'idx_spend', 'transactions', 'cardholders']
month_group = ['cardholder_issuing_country', 'month']
group_list = [ 'cardholder_issuing_country']

df_sn_int['year'] = df_sn_int['date_time'].dt.year
df_sn_int['month'] = df_sn_int['date_time'].dt.month

for i in metrics:
    # calc year-on-year differences
    df_sn_int[f'yoy_{i}'] = df_sn_int.groupby(month_group)[f'{i}'].diff(periods=1)

    # calc year-on-year % change
    df_sn_int[f'yoy_{i}_perc'] = df_sn_int.groupby(month_group)[f'{i}'].pct_change(periods=1)*100

    # index to jan 2019
    df_sn_int[f"index_{i}"] = df_sn_int.groupby(group_list)[f"{i}"].transform(
    lambda x: x / x.iloc[0]
)

In [ ]:
fig = px.line(
df_sn_int,

x="date_time",
y="index_idx_spend",
color = 'cardholder_issuing_country',
title=f"Index card-adj spend at postal districts",
height = 500,
template = 'simple_white',
color_discrete_map=sn_int_colours

)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.update_layout(legend_title_text=f'Venue postal district:', 
                  title_subtitle_text=f'MCG = All',
                  yaxis_title = f'Index sum international spend',
                 xaxis_title = 'Date')
fig.update_traces(selector=dict(name='UK'), line=dict(dash='dash'))  #dashed line for UK

fig = add_nations_periods(fig)


fig.show()

-----------------

-----------------

## Holyhead 

In [ ]:
holyhead_colours = {'LL65' : '#206095', # ocean blue
                    'Wider Anglesey': '#A8BD3A', # spring green
                    'Rest of UK' : '#871A5B', # beetroot purple
                    'Rest of International' : 'dimgrey', 
                    'Northern Ireland': '#746CB1', # lavender purple
                    'Republic of Ireland': '#118C7B',
                   'Holyhead': '#206095', # ocean blue
                   'Pembroke' :'#A8BD3A', # spring green
                   'Fishguard' : '#746CB1',
                   'Liverpool Birkenhead': '#118C7B'}

holyhead_districts = ['LL33'] + [ f'LL{i}' for i in range(54, 79)]



In [ ]:
def add_holyhead_closure(fig):

    concert_periods = [
        ("2024-11-20", "2025-01-15"),
    ]
    fillcolor = "gainsboro"

    fig.add_trace(
        go.Scatter(
            x=[None],
            y=[None],
            mode="markers",
            marker=dict(color=fillcolor, symbol="square"),
            name="Port closure",
        )
    )

    for start_date, end_date in concert_periods:
        fig.add_shape(
            type="rect",
            xref="x",
            yref="paper",
            x0=start_date,
            y0=0,
            x1=end_date,
            y1=1,
            fillcolor=fillcolor,
            opacity=0.5,
            layer="below",
            line_width=0,
        )

    return fig

### High-level

In [ ]:
client = bigquery.Client()

sql = f"""SELECT time_period_value, merchant_location, spend, transactions, cardholders
  FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
  WHERE time_period = 'Month' AND 
  merchant_location_level = 'POSTAL_DISTRICT' AND
  cardholder_issuing_level = 'All' AND
  mcg = 'All' AND
  time_period_value <= '202503'
  ORDER BY time_period_value, merchant_location
  """

hp_full = client.query(sql).to_dataframe()
hp_full = t.create_date_time(hp_full)

# removing non-district locations 
hp_full["number"] = hp_full["merchant_location"].str.extract(
    "(\d+)", expand=False
)
hp_full = hp_full.loc[hp_full["number"].isna() == False].drop(
    "number", axis=1
)

ll65 = hp_full[hp_full['merchant_location'] == 'LL65'].copy()
uk = hp_full.copy()

In [ ]:
ll65 = ll65.merge(sml_hl[['time_period_value', 'idx_cards']], on='time_period_value', how='left')
uk = uk.merge(sml_hl[['time_period_value', 'idx_cards']], on='time_period_value', how='left')


In [ ]:
# Adjust spend values for number of cardholders
dataframes = {'ll65': ll65, 'uk': uk}

for name, df in dataframes.items():
    df["idx_spend"] = df["spend"] / df["idx_cards"]


uk = (
        uk.groupby(["date_time"])
        .agg({"spend": "sum", "idx_spend" : "sum", "transactions": "sum", "cardholders": "sum"})
        .reset_index()
    )

uk['merchant_location'] = 'Rest of UK'

In [ ]:
def calc_index_metrics(df, group):
    df['year'] = df['date_time'].dt.year
    df['month'] = df['date_time'].dt.month

    month_group = group + ['month']

    df = df.sort_values(['date_time']).reset_index(drop=True)
    # index spend
    df["index_spend"] = df.groupby(group)["spend"].transform(
        lambda x: x / x.iloc[0]
    )
    df["index_spend_adj"] = df.groupby(group)["idx_spend"].transform(
        lambda x: x / x.iloc[0]
    )
    # index cardholders
    df["index_cardholders"] = df.groupby(group)["cardholders"].transform(
        lambda x: x / x.iloc[0]
    )
    # month-on-month
    df['mm_perc_spend'] = df.groupby(group)['index_spend'].pct_change(periods=1)*100
    df['mm_perc_spend_adj'] = df.groupby(group)['index_spend_adj'].pct_change(periods=1)*100
    df['mm_perc_cardholders'] = df.groupby(group)['index_cardholders'].pct_change(periods=1)*100

    # year-on-year
    df['yy_perc_spend'] = df.groupby(month_group)['index_spend'].pct_change(periods=1)*100
    df['yy_perc_spend_adj'] = df.groupby(month_group)['index_spend_adj'].pct_change(periods=1)*100

    df['yy_perc_cardholders'] = df.groupby(month_group)['index_cardholders'].pct_change(periods=1)*100
    
    return df

In [ ]:
ll65 = calc_index_metrics(ll65, group = ['merchant_location'])
uk = calc_index_metrics(uk, group = ['merchant_location'])


In [ ]:
all_dfs = pd.concat([ll65, uk])

In [ ]:
fig = px.line(
all_dfs,
x="date_time",
y="index_spend_adj",
color = 'merchant_location',
title=f"Index spend at areas",
height = 500,
template = 'simple_white',
color_discrete_map=holyhead_colours

)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.update_layout(legend_title_text=f'Merchant location:', 
                  title_subtitle_text=f'MCG = All',
                  yaxis_title = f'Index sum  spend',
                 xaxis_title = 'Date')
fig.update_traces(selector=dict(name='Rest of UK'), line=dict(dash='dash'))  #dashed line for UK

fig = add_covid_periods(fig)
fig = add_holyhead_closure(fig)

fig.show()

In [ ]:
fig = px.line(
all_dfs,
x="date_time",
y="yy_perc_spend_adj",
color = 'merchant_location',
height = 500,
template = 'simple_white',
color_discrete_map=holyhead_colours,
title = 'YY change card-adj'
)
fig = add_covid_periods(fig)


fig.show()

### Irish spend

In [ ]:
client = bigquery.Client()

sql_ire = f"""SELECT time_period_value, spend, transactions, cardholders
  FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
  WHERE time_period = 'Month' AND 
  merchant_location = 'LL65' AND
  cardholder_issuing_country = 'REPUBLIC OF IRELAND' AND
  mcg = 'All' AND
  time_period_value >= '202210' AND
  time_period_value <= '202503'
  ORDER BY time_period_value
  """

df_ire = client.query(sql_ire).to_dataframe()
df_ire = t.create_date_time(df_ire)
df_ire['cardholder_location'] = 'REPUBLIC OF IRELAND'

sql_bt = f"""SELECT time_period_value, cardholder_location, spend, transactions, cardholders
  FROM ons-fintrans-data-prod.fintrans_visa.retail_performance_high_streets_towns
  WHERE time_period = 'Month' AND 
  merchant_location = 'LL65' AND
  cardholder_location = 'BT' AND
  mcg = 'All' AND
  time_period_value >= '202201' AND
  time_period_value <= '202503'
  ORDER BY time_period_value, cardholder_location
  """

df_bt = client.query(sql_bt).to_dataframe()
df_bt = t.create_date_time(df_bt)

In [ ]:
irish_df = pd.concat([df_bt, df_ire]).reset_index(drop = True)
irish_df = irish_df.merge(int_hl[['time_period_value', 'idx_cards']], on='time_period_value', how='left')
# Adjust spend values for number of cardholders - using UK international
irish_df["idx_spend"] = irish_df["spend"] / irish_df["idx_cards"]

In [ ]:
# Adjust spend by country-specific cardholder numbers
irish_df["idx_cards"] = irish_df.groupby(['cardholder_location'])["cardholders"].transform(lambda x: x / x.iloc[0])
irish_df["idx_spend"] = irish_df["spend"] / irish_df["idx_cards"]

In [ ]:
irish_df = calc_index_metrics(irish_df, group = ['cardholder_location'])

In [ ]:
irish_df['cardholder_location'] =irish_df['cardholder_location'].replace({'BT': 'Northern Ireland', 'REPUBLIC OF IRELAND' : 'Republic of Ireland'})

In [ ]:
fig = px.line(
irish_df,
x="date_time",
y="index_spend",
color = 'cardholder_location',
title=f"Index Irish spend",
height = 500,
template = 'simple_white',
color_discrete_map=holyhead_colours

)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.update_layout(legend_title_text=f'Origin of cardholder:', 
                  title_subtitle_text=f'MCG = All',
                  yaxis_title = f'Index sum  spend',
                 xaxis_title = 'Date')
fig.update_traces(selector=dict(name='UK'), line=dict(dash='dash'))  #dashed line for UK
fig = add_holyhead_closure(fig)

fig.show()

In [ ]:
fig = px.line(
irish_df,
x="date_time",
y="index_spend",
color = 'cardholder_location',
title=f"Index Irish spend",
height = 500,
template = 'simple_white',
color_discrete_map=holyhead_colours

)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.update_layout(legend_title_text=f'Origin of cardholder:', 
                  title_subtitle_text=f'MCG = All',
                  yaxis_title = f'Index sum  spend',
                 xaxis_title = 'Date')
fig.update_traces(selector=dict(name='UK'), line=dict(dash='dash'))  #dashed line for UK
fig = add_holyhead_closure(fig)

fig.show()

In [ ]:
fig = px.line(
irish_df,
x="date_time",
y="yy_perc_spend_adj",
color = 'cardholder_location',
title=f"YY",
height = 500,
template = 'simple_white',
color_discrete_map=holyhead_colours

)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.update_layout(legend_title_text=f'Origin of cardholder:', 
                  title_subtitle_text=f'MCG = All',
                  yaxis_title = f'Index sum  spend',
                 xaxis_title = 'Date')
fig.update_traces(selector=dict(name='UK'), line=dict(dash='dash'))  #dashed line for UK
fig = add_holyhead_closure(fig)

fig.show()

### Spend at other ports

In [ ]:
other_ports = ('CH41', 'SA64',  'LL65')
port_colours = {'Holyhead': '#A8BD3A',
                   'Fishguard' : '#746CB1',
                   'Birkenhead': '#F66068'}

client = bigquery.Client()

sql_nearby = f"""SELECT time_period_value, merchant_location, spend, transactions, cardholders
  FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
  WHERE time_period = 'Month' AND 
  merchant_location_level = 'POSTAL_DISTRICT' AND
  merchant_location IN {other_ports} AND
  cardholder_issuing_country = 'REPUBLIC OF IRELAND' AND
  mcg = 'All' AND
  time_period_value >= '202111' AND
  time_period_value <= '202503'
  ORDER BY time_period_value, merchant_location
  """

df_near = client.query(sql_nearby).to_dataframe()
df_near = t.create_date_time(df_near)


df_near['merchant_location'] =df_near['merchant_location'].replace({'CH41': 'Birkenhead', 
                                                                    'LL65' : 'Holyhead',
                                                                   'SA64' : 'Fishguard'})

In [ ]:
df_near = df_near.merge(int_hl[['time_period_value', 'idx_cards']], on='time_period_value', how='left')
# Adjust spend values for number of cardholders
df_near["idx_spend"] = df_near["spend"] / df_near["idx_cards"]

In [ ]:
df_near = calc_index_metrics(df_near, group = ['merchant_location'])

In [ ]:
fig = px.line(
df_near,
x="date_time",
y="yy_perc_spend_adj",
color = 'merchant_location',
    color_discrete_map=port_colours, 
template='simple_white',
height = 500,
width = 800)

fig.show()

In [ ]:
df_near

In [ ]:
fig = px.line(
df_near,
x="date_time",
y="yy_perc_spend_adj",
color = 'merchant_location',
    color_discrete_map=port_colours, 
template='simple_white',
height = 500,
width = 800)

fig.show()

In [ ]:
df_near[df_near['merchant_location'] == 'Fishguard']